In [1]:
import sys
from check_calibration.ProbabilityIntegralTransfer import prob_integ_xfer
import numpy as np
from pprint import pprint
import matplotlib
import matplotlib.pyplot as plt
import os
import pickle as pk
import random

import os, sys
import check_calibration.utils as utils
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score
from concurrent.futures import ProcessPoolExecutor

In [2]:

def rf(X_vir, y_vir, X_real, y_real, save, proportion, portion_vir, exp_protocol
       , model_mode, frame_size_seconds, step_size_seconds):

#     print(X_vir.shape)
#     print(X_real.shape)

    '''
    Sliding Window
    '''

#     frame_size_seconds = 1
#     step_size_seconds = 0.5
    ecdf_num = 15
    sampling_rate = 20
    n_estimators = 25

    frame_size = int(frame_size_seconds * sampling_rate)
    step_size = int(step_size_seconds * sampling_rate)
    
    
    ''' vir sliding window'''
    X_vir_win = utils.sliding_window(X_vir,
          (frame_size, X_vir.shape[1]),
          (step_size, 1))

    y_vir_win = np.asarray([
      [i[-1]] for i in utils.sliding_window(
        y_vir, 
        frame_size, 
        step_size)])
    y_vir_win = y_vir_win.reshape(len(y_vir_win)).astype(np.uint8)
    
    
    ''' vir feature extraction'''
    
    _feat = utils.ecdfRep(X_vir_win[0], components=ecdf_num)
    X_vir_feat = np.empty((X_vir_win.shape[0], _feat.shape[0]))
    for i in range(X_vir_win.shape[0]):
      X_vir_feat[i] = utils.ecdfRep(X_vir_win[i], components=ecdf_num)
    
    
    
# randomly select percent of vir data 
#     portion_vir = 1
    ys = np.unique(y_vir_win)

    X_real_por = np.empty(X_vir_feat.shape)
    y_real_por = np.empty(y_vir_win.shape)

    i_start = 0
    for y in ys:
      idx_y = y_vir_win == y
      num_por = int(np.sum(idx_y)*portion_vir)

      X_real_y = X_vir_feat[idx_y]
      y_real_y = y_vir_win[idx_y]

      #shuffle the array so it contains data from all participant 

      random.shuffle(X_real_y)
      random.shuffle(y_real_y)  

      X_real_por[i_start:i_start+num_por] = X_real_y[:num_por]
      y_real_por[i_start:i_start+num_por] = y_real_y[:num_por]

      i_start += num_por

    X_vir_feat = X_real_por[:i_start]
    y_vir_win = y_real_por[:i_start]

    print (f'portion {portion_vir}:', X_vir_feat.shape, y_vir_win.shape)

    
    list_f1_score = []
    #leave 1 out cross val
    list_acc = []
    f1_per_cls = np.zeros((len(np.unique(y_real[0])), len(y_real)))
    print(f1_per_cls.shape)
    idx =0
    for per in range(len(X_real)):
        X_real_test = X_real[per]
        y_real_test = y_real[per]
        
        
        #concatenate all training data from all training subjects 
        X_real_train = None
        y_real_train = None
        for j in range(len(X_real)):
            if j == per:
                    continue 
            for cls_idx in np.unique(y_real[0]):
                # get index of cls in a subject
                cls_idx_all = np.where(y_real[j] == cls_idx)[0]
                X_real_cls = X_real[j][cls_idx_all]
                y_real_cls = y_real[j][cls_idx_all]
                
                if X_real_train is None:
                    X_real_train = X_real_cls
                    y_real_train = y_real_cls
                else:
                    X_real_train = np.vstack((X_real_train, X_real_cls))
                    y_real_train = np.concatenate((y_real_train, y_real_cls))

        
        X_copy = X_real_train.copy()
        y_copy = y_real_train.copy()
        X_real_train = None
        y_real_train = None
        for cls_idx in np.unique(y_real[0]):
            cls_idx_all = np.where(y_copy == cls_idx)[0]
            X_real_cls = X_copy[cls_idx_all]
            y_real_cls = y_copy[cls_idx_all]

            if X_real_train is None:
                X_real_train = X_real_cls
                y_real_train = y_real_cls
            else:
                X_real_train = np.vstack((X_real_train, X_real_cls))
                y_real_train = np.concatenate((y_real_train, y_real_cls))
#        print(y_real_train)

        ''' real sliding window'''
        X_real_win_train = utils.sliding_window(X_real_train,
              (frame_size, X_real_train.shape[1]),
              (step_size, 1))

        y_real_win_train = np.asarray([
          [i[-1]] for i in utils.sliding_window(
            y_real_train, 
            frame_size, 
            step_size)])
        y_real_win_train = y_real_win_train.reshape(len(y_real_win_train)).astype(np.uint8)
        
        "real test"
        X_real_win_test = utils.sliding_window(X_real_test,
              (frame_size, X_real_test.shape[1]),
              (step_size, 1))

        y_real_win_test = np.asarray([
          [i[-1]] for i in utils.sliding_window(
            y_real_test, 
            frame_size, 
            step_size)])
        y_real_win_test = y_real_win_test.reshape(len(y_real_win_test)).astype(np.uint8)

        ''' real feature extraction'''
        _feat = utils.ecdfRep(X_real_win_train[0], components=ecdf_num)
        X_real_feat_train = np.empty((X_real_win_train.shape[0], _feat.shape[0]))
        for i in range(X_real_win_train.shape[0]):
            X_real_feat_train[i] = utils.ecdfRep(X_real_win_train[i], components=ecdf_num)
            
        _feat = utils.ecdfRep(X_real_win_test[0], components=ecdf_num)
        X_real_feat_test = np.empty((X_real_win_test.shape[0], _feat.shape[0]))
        for i in range(X_real_win_test.shape[0]):
            X_real_feat_test[i] = utils.ecdfRep(X_real_win_test[i], components=ecdf_num)

        '''
        Experiment
        '''
        #for i in range(len(portion_val)):
        portion = proportion

        #exp_protocol = 'real' # real / vir / real+vir

        X_real_train, y_real_train = X_real_feat_train, y_real_win_train
        X_real_test, y_real_test = X_real_feat_test, y_real_win_test
        
        if model_mode == '_complex':
            clf = RandomForestClassifier(n_estimators=n_estimators)
        elif model_mode == '_simple':
            clf = RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_split=5, min_samples_leaf=2, 
                                         max_features='sqrt', max_leaf_nodes=10)
        else: 
            assert False, model_mode

        ''' portion real '''
        if portion == 1.:
            X_real_por = X_real_train
            y_real_por = y_real_train
        else:
            ys = np.unique(y_real_train)

            X_real_por = np.empty(X_real_train.shape)
            y_real_por = np.empty(y_real_train.shape)

            i_start = 0
            for y in ys:
              idx_y = y_real_train == y
              num_por = int(np.sum(idx_y)*portion)

              X_real_y = X_real_train[idx_y]
              y_real_y = y_real_train[idx_y]
                
              #shuffle the array so it contains data from all participant 
            
              random.shuffle(X_real_y)
              random.shuffle(y_real_y)  
                
              X_real_por[i_start:i_start+num_por] = X_real_y[:num_por]
              y_real_por[i_start:i_start+num_por] = y_real_y[:num_por]

              i_start += num_por

            X_real_por = X_real_por[:i_start]
            y_real_por = y_real_por[:i_start]

        print (f'portion {portion}:', X_real_por.shape, y_real_por.shape)

        print (exp_protocol)
        if exp_protocol == 'real':
            ''' train only with real '''
            X_train = X_real_por
            y_train = y_real_por
        elif exp_protocol == 'vir':
            ''' train only with vir '''
            X_train = X_vir_feat.copy()
            y_train = y_vir_win.copy()
        elif exp_protocol == 'real+vir':
            X_train = np.concatenate([X_real_por, X_vir_feat])
            y_train = np.concatenate([y_real_por, y_vir_win])
        else:
            assert False, exp_protocol

        ''' standardize '''
        X_mean = np.mean(X_train, axis=0).reshape((1,-1))
        X_std = np.mean(X_train, axis=0).reshape((1,-1))
        X_train -= X_mean
        X_train /= X_std
        X_real_test -= X_mean
        X_real_test /= X_std

        clf.fit(X_train,y_train)

        pred_test = clf.predict(X_real_test)

        f1 = f1_score(y_real_test, pred_test, average='macro')
        report = classification_report(y_real_test, pred_test, output_dict=True)
        print(f1)
        #training f1
        training_f1 = f1_score(y_train, clf.predict(X_train), average='macro')
        print(f'training f1: {training_f1}')
        
        acc = accuracy_score(y_real_test, pred_test)
#         print(acc)  
#         print(report.keys())
        labels = list(report.keys())[:-3]
        
        for i in (labels):
# #             print(int(i))
#             if f'{i}' in report.keys():
#                 f1_per_cls[int(i)][idx] += report[f'{i}']['f1-score']
#             else:
#                 print(int(i))
             f1_per_cls[int(float(i))][idx] +=  report[i]['f1-score']
        list_acc.append(acc)
        list_f1_score.append(f1)
        idx +=1

    print(list_f1_score)
    print (np.mean(list_f1_score))

    return np.array(list_f1_score), f1_per_cls

In [56]:
#load real and vir data 

# sub_array = np.array(([0,1,-1])).astype(int)
loc = 'all_loc'
data_dir = 'data/usc-had_50'
ik_mode = "_ik"
noise_mode = ''
model_mode = '_complex'

portion_val = np.arange(11)/10
portion_val[0] = 0.05
result_low = np.zeros(11)
save = '/f1_per_cls_mix_low_0.1.txt'

file_vir = f'{data_dir}/resized_{loc}_cal{ik_mode}{noise_mode}.npz'
# file_vir = f'data/realworld_50/resized_{loc}_cal{ik_mode}{noise_mode}.npz'
Xy_vir = np.load(file_vir)
X_vir = Xy_vir['x']
y_vir = Xy_vir['y'].astype(int)
print(y_vir.shape)

Xy_real = pk.load(open(f"{data_dir}/real_{loc}.p",'rb'))
# Xy_real = pk.load(open(f"{data_dir}/real_{loc}_sync.p",'rb'))
X_real = Xy_real[0]
y_real = Xy_real[1]

print(len(X_real))

for i in range(len(X_real)):
    X_real[i] = np.array(X_real[i])
    y_real[i] = np.array(y_real[i]).astype(int)
#     print(X_real[i].shape)
#     print(np.unique(y_real[i]))
# X_real = [X_real[i] for i in sub_array]
# y_real = [y_real[i] for i in sub_array]   
print(len(X_real))
    

(82875,)
14
14


In [23]:
# run leave one out experiment
result_dir = 'recognition_result/usc-had_50'
os.makedirs(result_dir, exist_ok = True)
    
total_folds = len(y_real)
portion_val = [0.6]
num_trials = 3
proportion = 1
prop_vir = 1
frame_size_seconds = 2
step_size_seconds = 1
print(total_folds)

list_f1 = []
result_all_vir = np.zeros((num_trials,total_folds))
f1_per_class = None

for i in range(num_trials):
    result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, proportion, prop_vir, 'real', model_mode,
                                    frame_size_seconds, step_size_seconds)
    list_f1.append(np.mean(result_all_vir[i]))
    
    if f1_per_class is None:
        f1_per_class = f1_trial
    else:
        f1_per_class = np.hstack((f1_per_class, f1_trial))

print(f1_per_class.shape)
print(np.mean(np.array(list_f1)))
print(np.mean(result_all_vir, axis =0))
# np.save(f'{result_dir}/real_{loc}_{proportion}_per_class{model_mode}.npy', f1_per_class)

# list_f1 = []
# result_all_vir = np.zeros((num_trials,total_folds))
# f1_per_class = None
# save = ""
# for i in range(num_trials):
#     result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, proportion, prop_vir, 'real+vir',model_mode,
#                                    frame_size_seconds, step_size_seconds)
#     list_f1.append(np.mean(result_all_vir[i]))
    
#     if f1_per_class is None:
#         f1_per_class = f1_trial
#     else:
#         f1_per_class = np.hstack((f1_per_class, f1_trial))
            
# print(np.mean(np.array(list_f1)))
# print(np.mean(result_all_vir, axis =0))
# np.save(f'{result_dir}/mixed_{loc}_{proportion}_per_class{ik_mode}{noise_mode}{model_mode}.npy', f1_per_class)

# list_f1 = []
# result_all_vir = np.zeros((num_trials,total_folds))
# f1_per_class = None
# save = ""
# for i in range(num_trials):
#     result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, proportion, prop_vir, 'vir', model_mode,
#                                    frame_size_seconds, step_size_seconds)
#     list_f1.append(np.mean(result_all_vir[i]))
    
#     if f1_per_class is None:
#         f1_per_class = f1_trial
#     else:
#         f1_per_class = np.hstack((f1_per_class, f1_trial))
            
# print(np.mean(np.array(list_f1)))
# print(np.mean(result_all_vir, axis =0))
# np.save(f'{result_dir}/virtual_{loc}_{proportion}_per_class{ik_mode}{noise_mode}{model_mode}.npy', f1_per_class)

15
portion 1: (2466, 336) (2466,)
(1, 15)
portion 1: (9084, 336) (9084,)
real
1.0
training f1: 1.0


IndexError: index 7 is out of bounds for axis 0 with size 1

In [8]:
# run leave one out experiment but with subsets of proportions of virtual data 
result_dir = 'recognition_result/realworld_1000/vary_vir'
os.makedirs(result_dir, exist_ok = True)
    
total_folds = len(y_real)
portion_val = [0.6]
num_trials = 3
proportion = 1
frame_size_seconds = 2
step_size_seconds = 1
print(total_folds)

vir_proportions = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

for prop in vir_proportions:
    #mixed
    save = f'{result_dir}/virtual_{loc}_{prop}_per_class{ik_mode}{noise_mode}{model_mode}.npy'
    if os.path.exists(save):
        print(f'{save} exists')
        continue
    
#     list_f1 = []
#     result_all_vir = np.zeros((num_trials,total_folds))
#     f1_per_class = None
#     for i in range(num_trials):
#         result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, proportion, prop, 'real+vir', model_mode,
#                                        frame_size_seconds, step_size_seconds)
#         list_f1.append(np.mean(result_all_vir[i]))

#         if f1_per_class is None:
#             f1_per_class = f1_trial
#         else:
#             f1_per_class = np.hstack((f1_per_class, f1_trial))

#     print(np.mean(np.array(list_f1)))
#     print(np.mean(result_all_vir, axis =0))
#     np.save(save, f1_per_class)
    
    
    #virtual 
    save = f'{result_dir}/virtual_{loc}_{prop}_per_class{ik_mode}{noise_mode}{model_mode}.npy'
    if os.path.exists(save):
        print(f'{save} exists')
        continue
    
    list_f1 = []
    result_all_vir = np.zeros((num_trials,total_folds))
    f1_per_class = None
    for i in range(num_trials):
        result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, proportion, prop, 'vir', model_mode,
                                       frame_size_seconds, step_size_seconds)
        list_f1.append(np.mean(result_all_vir[i]))

        if f1_per_class is None:
            f1_per_class = f1_trial
        else:
            f1_per_class = np.hstack((f1_per_class, f1_trial))

    print(np.mean(np.array(list_f1)))
    print(np.mean(result_all_vir, axis =0))
    np.save(save, f1_per_class)

15
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.001_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.005_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.01_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.05_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.1_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.2_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.3_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.4_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.5_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vir/virtual_all_loc_0.6_per_class_ik_complex.npy exists
recognition_result/realworld_1000/vary_vi

D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8113857517786665
training f1: 1.0
portion 1: (61853, 336) (61853,)
vir
0.7904951019344597
training f1: 1.0
portion 1: (61296, 336) (61296,)
vir
0.7759879153699677
training f1: 1.0
portion 1: (61489, 336) (61489,)
vir
0.7554718566413201
training f1: 1.0
portion 1: (62182, 336) (62182,)
vir


D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6821349492741278
training f1: 1.0
portion 1: (61797, 336) (61797,)
vir
0.860125621810715
training f1: 1.0
portion 1: (61529, 336) (61529,)
vir
0.7223083057339486
training f1: 1.0
portion 1: (61917, 336) (61917,)
vir
0.8010018588782855
training f1: 1.0
portion 1: (62201, 336) (62201,)
vir
0.6899158202921113
training f1: 1.0
portion 1: (62000, 336) (62000,)
vir
0.6487016685665535
training f1: 1.0
portion 1: (62157, 336) (62157,)
vir
0.8296480061557057
training f1: 1.0
portion 1: (62033, 336) (62033,)
vir
0.6406629301404227
training f1: 1.0
portion 1: (62112, 336) (62112,)
vir
0.6776627221644571
training f1: 1.0
portion 1: (61963, 336) (61963,)
vir
0.6124991728930258
training f1: 1.0
[0.7292942241608598, 0.8113857517786665, 0.7904951019344597, 0.7759879153699677, 0.7554718566413201, 0.6821349492741278, 0.860125621810715, 0.7223083057339486, 0.8010018588782855, 0.6899158202921113, 0.6487016685665535, 0.8296480061557057, 0.6406629301404227, 0.6776627221644571, 0.6124991728930258]
0.735153

...

0.66866491123246
training f1: 1.0
portion 1: (61797, 336) (61797,)
vir
0.8984049509221496
training f1: 1.0
portion 1: (61529, 336) (61529,)
vir
0.7531008201175005
training f1: 1.0
portion 1: (61917, 336) (61917,)
vir
0.7568629755298508
training f1: 1.0
portion 1: (62201, 336) (62201,)
vir
0.711505885015562
training f1: 1.0
portion 1: (62000, 336) (62000,)
vir
0.592991374236173
training f1: 1.0
portion 1: (62157, 336) (62157,)
vir
0.8279490019464553
training f1: 1.0
portion 1: (62033, 336) (62033,)
vir
0.6061324729580335
training f1: 1.0
portion 1: (62112, 336) (62112,)
vir
0.7441360557434259
training f1: 1.0
portion 1: (61963, 336) (61963,)
vir
0.6294504468970541
training f1: 1.0
[0.7350273753382647, 0.7862964110812651, 0.8372799409321602, 0.7820044198981291, 0.7464251395686359, 0.66866491123246, 0.8984049509221496, 0.7531008201175005, 0.7568629755298508, 0.711505885015562, 0.592991374236173, 0.8279490019464553, 0.6061324729580335, 0.7441360557434259, 0.6294504468970541]
0.738415478761

D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.782873309018632
training f1: 1.0
portion 1: (61853, 336) (61853,)
vir
0.7650862304778714
training f1: 1.0
portion 1: (61296, 336) (61296,)
vir
0.7839949775318673
training f1: 1.0
portion 1: (61489, 336) (61489,)
vir
0.7188035842774236
training f1: 1.0
portion 1: (62182, 336) (62182,)
vir


D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6959859123203652
training f1: 1.0
portion 1: (61797, 336) (61797,)
vir
0.8400611422039733
training f1: 1.0
portion 1: (61529, 336) (61529,)
vir
0.706106594566289
training f1: 1.0
portion 1: (61917, 336) (61917,)
vir
0.7661632940713545
training f1: 1.0
portion 1: (62201, 336) (62201,)
vir
0.6791181810445878
training f1: 1.0
portion 1: (62000, 336) (62000,)
vir
0.6164764587579965
training f1: 1.0
portion 1: (62157, 336) (62157,)
vir
0.7663452955751002
training f1: 1.0
portion 1: (62033, 336) (62033,)
vir
0.7075377158907601
training f1: 1.0
portion 1: (62112, 336) (62112,)
vir
0.7081988985723746
training f1: 1.0
portion 1: (61963, 336) (61963,)
vir
0.6292968365082844
training f1: 1.0
[0.7180206997902223, 0.782873309018632, 0.7650862304778714, 0.7839949775318673, 0.7188035842774236, 0.6959859123203652, 0.8400611422039733, 0.706106594566289, 0.7661632940713545, 0.6791181810445878, 0.6164764587579965, 0.7663452955751002, 0.7075377158907601, 0.7081988985723746, 0.6292968365082844]
0.7256046

In [58]:
# run leave one out experiment but with subsets of proportions of real data 
result_dir = 'recognition_result/usc-had_50/vary_real'
os.makedirs(result_dir, exist_ok = True)
    
total_folds = len(y_real)
portion_val = [0.6]
num_trials = 3
proportion = 1
prop_vir = 1
frame_size_seconds = 2
step_size_seconds = 1
print(total_folds)

# real_proportions = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
real_proportions = [0.02]

for prop in real_proportions:
    #mixed
#     save = f'{result_dir}/virtual_{loc}_{prop}_per_class{ik_mode}{noise_mode}{model_mode}.npy'
    if os.path.exists(save):
        print(f'{save} exists')
        continue
    
    list_f1 = []
    result_all_vir = np.zeros((num_trials,total_folds))
    f1_per_class = None

    for i in range(num_trials):
        result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, prop, prop_vir, 'real', model_mode,
                                        frame_size_seconds, step_size_seconds)
        list_f1.append(np.mean(result_all_vir[i]))

        if f1_per_class is None:
            f1_per_class = f1_trial
        else:
            f1_per_class = np.hstack((f1_per_class, f1_trial))

    print(f1_per_class.shape)
    print(np.mean(np.array(list_f1)))
    print(np.mean(result_all_vir, axis =0))
    np.save(f'{result_dir}/real_{loc}_{prop}_per_class{model_mode}.npy', f1_per_class)

    list_f1 = []
    result_all_vir = np.zeros((num_trials,total_folds))
    f1_per_class = None
    save = ""
    for i in range(num_trials):
        result_all_vir[i], f1_trial= rf(X_vir, y_vir, X_real, y_real, save, prop, prop_vir, 'real+vir',model_mode,
                                       frame_size_seconds, step_size_seconds)
        list_f1.append(np.mean(result_all_vir[i]))

        if f1_per_class is None:
            f1_per_class = f1_trial
        else:
            f1_per_class = np.hstack((f1_per_class, f1_trial))

    print(np.mean(np.array(list_f1)))
    print(np.mean(result_all_vir, axis =0))
    np.save(f'{result_dir}/mixed_{loc}_{prop}_per_class{ik_mode}{noise_mode}{model_mode}.npy', f1_per_class)

14
portion 1: (4142, 48) (4142,)
(12, 14)
portion 0.02: (519, 48) (519,)
real
0.3988672450093372
training f1: 1.0
portion 0.02: (523, 48) (523,)
real
0.32082702677006725
training f1: 1.0


D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


portion 0.02: (529, 48) (529,)
real
0.3904237952045784
training f1: 1.0


D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


portion 0.02: (525, 48) (525,)
real
0.3173359959641829
training f1: 1.0
portion 0.02: (529, 48) (529,)
real
0.39725141860475555
training f1: 1.0


...

0.3572685534628705
training f1: 1.0
portion 0.02: (504, 48) (504,)
real+vir
0.4108342877225059
training f1: 0.9997882198573889
portion 0.02: (508, 48) (508,)
real+vir
0.4595376053604938
training f1: 1.0
portion 0.02: (505, 48) (505,)
real+vir
0.447148201044766
training f1: 1.0
portion 0.02: (503, 48) (503,)
real+vir
0.55004982540943
training f1: 1.0
portion 0.02: (506, 48) (506,)
real+vir
0.43309036344572177
training f1: 1.0
portion 0.02: (515, 48) (515,)
real+vir
0.292348479348383
training f1: 0.999795212397952
[0.49082075965840183, 0.4753391929217187, 0.5979681658571266, 0.5315813307343066, 0.48573085720980186, 0.5081645325622571, 0.41630109920109315, 0.3572685534628705, 0.4108342877225059, 0.4595376053604938, 0.447148201044766, 0.55004982540943, 0.43309036344572177, 0.292348479348383]
0.46115594670991983
portion 1: (4142, 48) (4142,)
(12, 14)
portion 0.02: (519, 48) (519,)
real+vir
0.5115652608030046
training f1: 1.0
portion 0.02: (523, 48) (523,)
real+vir
0.465958772237219
training

D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.33078687684777924
training f1: 1.0
portion 0.02: (504, 48) (504,)
real+vir
0.45819256784091844
training f1: 1.0
portion 0.02: (508, 48) (508,)
real+vir
0.4912161317380053
training f1: 0.999809371207236
portion 0.02: (505, 48) (505,)
real+vir
0.3664791746626457
training f1: 1.0
portion 0.02: (503, 48) (503,)
real+vir
0.5276308271416817
training f1: 1.0
portion 0.02: (506, 48) (506,)
real+vir
0.43034736929118006
training f1: 0.9997887607888621
portion 0.02: (515, 48) (515,)
real+vir
0.31597702184065546
training f1: 0.9997899527665003
[0.5115652608030046, 0.465958772237219, 0.5902629511541716, 0.5254528171955374, 0.5722631468233705, 0.5453220092672849, 0.419365791395178, 0.33078687684777924, 0.45819256784091844, 0.4912161317380053, 0.3664791746626457, 0.5276308271416817, 0.43034736929118006, 0.31597702184065546]
0.46791576558847375
portion 1: (4142, 48) (4142,)
(12, 14)
portion 0.02: (519, 48) (519,)
real+vir
0.5559251028597245
training f1: 1.0
portion 0.02: (523, 48) (523,)
real+vir
0.

D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Program\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3257868727864736
training f1: 0.9997954702302527
portion 0.02: (504, 48) (504,)
real+vir
0.46457662760575175
training f1: 0.999760720379277
portion 0.02: (508, 48) (508,)
real+vir
0.4990883196338079
training f1: 1.0
portion 0.02: (505, 48) (505,)
real+vir
0.4111800998228065
training f1: 1.0
portion 0.02: (503, 48) (503,)
real+vir
0.5402478558185123
training f1: 1.0
portion 0.02: (506, 48) (506,)
real+vir
0.4370274446714964
training f1: 1.0
portion 0.02: (515, 48) (515,)
real+vir
0.3655465290300837
training f1: 1.0
[0.5559251028597245, 0.4670452977923983, 0.6302849257014564, 0.5117714244046391, 0.4807118084116317, 0.4591650163146128, 0.40665755149981125, 0.3257868727864736, 0.46457662760575175, 0.4990883196338079, 0.4111800998228065, 0.5402478558185123, 0.4370274446714964, 0.3655465290300837]
0.46821534831094336
0.4657623535364457
[0.51943704 0.46944775 0.60617201 0.52293519 0.51290194 0.50421719
 0.41410815 0.33794743 0.44453449 0.48328069 0.40826916 0.5393095
 0.43348839 0.32462401]

In [44]:
#calculate macro f1 
locations = ['wrist', 'all_loc']
data = ['real','virtual', 'mixed']
proportion = [ '0.1', '1']

for loc in locations:
    for pro in proportion:
        for d in data:
        
            f1_per_cls = np.load(f'recognition_result/{d}_{loc}_{pro}_per_class.npy')
            mean = np.mean(f1_per_cls)
            per_f1 = np.mean(f1_per_cls, axis= 0).reshape(3,15)
            trial_f1 = np.mean(per_f1, axis= 1)
#             print(per_f1.shape)
#             print(trial_f1)
#             print(np.std(trial_f1))
#             print(np.std(per_f1))
#             print(f1_per_cls.shape)
            print(f'{d}_{loc}_{pro}: {mean} +- {np.std(trial_f1)}')
            print()
    

real_wrist_0.1: 0.44371222622855644 +- 0.004000585547892102

virtual_wrist_0.1: 0.5729824020594033 +- 0.0029933088095705834

mixed_wrist_0.1: 0.5885694412303891 +- 0.004570209070019517

real_wrist_1: 0.612805732214365 +- 0.006077514192361513

virtual_wrist_1: 0.5752004731832613 +- 0.00940299957654236

mixed_wrist_1: 0.623680829551123 +- 0.00579877070056614

real_all_loc_0.1: 0.4813624535386182 +- 0.008488684889517475

virtual_all_loc_0.1: 0.7522699404474195 +- 0.0032712580032467197

mixed_all_loc_0.1: 0.7756634781198112 +- 0.007597475768053443

real_all_loc_1: 0.7152858129431194 +- 0.010669841295236496

virtual_all_loc_1: 0.7568780661809273 +- 0.011224521510962808

mixed_all_loc_1: 0.7690427772509687 +- 0.0040594283401381775



In [40]:
#calculate f1 per class change
locations = ['wrist', 'all_loc']
data = ['real','virtual', 'mixed']
proportion = [ '0.1', '1']
activities = ['climb down stairs', 'climb up stairs', 'jumping', 'lying', 'running', 'sitting', 'standing' , 'walking' ]
for loc in locations:
    for pro in proportion:
        
            real_f1_per_cls = np.load(f'recognition_result/real_{loc}_{pro}_per_class.npy')
            mixed_f1_per_cls = np.load(f'recognition_result/mixed_{loc}_{pro}_per_class.npy')
            change_f1_per_cls = mixed_f1_per_cls - real_f1_per_cls
            class_f1_change = np.mean(change_f1_per_cls, axis= 1)
#             trial_f1 = np.mean(per_f1, axis= 1)
            print(f'{loc}_{pro}')
            print(class_f1_change)
#             print(trial_f1)
#             print(np.std(trial_f1))
#             print(np.std(per_f1))
#             print(f1_per_cls.shape)
#             print(f'{loc}_{pro}: {mean} +- {np.std(trial_f1)}')
#             print()
    

wrist_0.1
[ 0.08716264 -0.00130937  0.07347297  0.44749787  0.08634731  0.13806735
  0.18539668  0.14222226]
wrist_1
[ 0.0001181   0.01444382 -0.00340543  0.01711734 -0.00671653  0.03656803
  0.01145665  0.01741879]
all_loc_0.1
[0.45582651 0.1675652  0.18412023 0.12032266 0.1289279  0.32542449
 0.49556113 0.47666008]
all_loc_1
[0.0263525  0.02866955 0.04480148 0.06440886 0.06548793 0.10896529
 0.05804619 0.03332392]
